In [1]:
import pandas as pd
import os
import logging
import sys

### Load environment vars and directories

In [2]:
KEY_DIR = os.path.join(os.getenv("DOCUMENTS"), "govuk-network-data", "key")
KEY_PATH = os.path.join(KEY_DIR, os.listdir(KEY_DIR)[0])
PROJECT_ID = "govuk-bigquery-analytics"

#### Logging for `pandas_gbq`

In [3]:
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(stream=sys.stdout))

### Extract page-hit only user journeys for February 11 to 18
8.8 GB

In [22]:
query = """SELECT
  COUNT(*) AS Occurrences,
  PageSeq_Length,
  PageSequence
FROM (
  SELECT
    *
  FROM (
    SELECT
      CONCAT(fullVisitorId,"-",CAST(visitId AS STRING),"-",CAST(visitNumber AS STRING)) AS sessionId,
      STRING_AGG(CONCAT(pagePath,"<<",CONCAT(htype,"<:<",IFNULL(eventCategory,
              "NULL"),"<:<",IFNULL(eventAction,
              "NULL")),"<<",taxon), ">>") OVER (PARTITION BY fullVisitorId, visitId ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS Sequence,
      STRING_AGG(IF(htype = 'PAGE',
          pagePath,
          NULL),">>") OVER (PARTITION BY fullVisitorId, visitId ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSequence,
      SUM(IF(htype='PAGE',
          1,
          0)) OVER (PARTITION BY fullVisitorId, visitId ORDER BY hitNumber ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING ) AS PageSeq_Length
    FROM (
      SELECT
        fullVisitorId,
        visitId,
        visitNumber,
        visitStartTime,
        hits.page.pagePath AS pagePath,
        hits.hitNumber AS hitNumber,
        hits.type AS htype,
        hits.eventInfo.eventAction AS eventAction,
        hits.eventInfo.eventCategory AS eventCategory,
        (
        SELECT
          value
        FROM
          hits.customDimensions
        WHERE
          index=59) AS taxon
      FROM
        `govuk-bigquery-analytics.87773428.ga_sessions_*` AS sessions
      CROSS JOIN
        UNNEST(sessions.hits) AS hits
      WHERE
        _TABLE_SUFFIX BETWEEN '20190302'
        AND '20190330' ) )
  WHERE
    PageSeq_Length > 1
    AND REGEXP_CONTAINS(Sequence, r"d7bdaee2-8ea5-460e-b00d-6e9382eb6b61|d6c2de5d-ef90-45d1-82d4-5f2438369eea|58adb819-5a65-4897-a790-5b7925c567e7|06e2928c-57b1-4b8d-a06e-3dde9ce63a6f")
  GROUP BY
    sessionId,
    Sequence,
    PageSequence,
    PageSeq_Length)
GROUP BY
  PageSequence,
  PageSeq_Length
"""

### Extract data from BigQuery

In [23]:
df_in = pd.read_gbq(query,
                       project_id=PROJECT_ID,
                       reauth=False,
                       private_key=KEY_PATH,
                       dialect="standard")

Requesting query... 
Query running...
Job ID: 803cfef7-1043-4707-9ecc-60ab46d7630a
  Elapsed 7.21 s. Waiting...
  Elapsed 8.47 s. Waiting...
  Elapsed 9.84 s. Waiting...
  Elapsed 11.13 s. Waiting...
  Elapsed 12.5 s. Waiting...
  Elapsed 13.83 s. Waiting...
  Elapsed 15.16 s. Waiting...
  Elapsed 16.49 s. Waiting...
  Elapsed 17.82 s. Waiting...
  Elapsed 19.15 s. Waiting...
  Elapsed 20.49 s. Waiting...
  Elapsed 21.82 s. Waiting...
  Elapsed 23.15 s. Waiting...
  Elapsed 24.48 s. Waiting...
  Elapsed 25.74 s. Waiting...
  Elapsed 27.02 s. Waiting...
  Elapsed 28.29 s. Waiting...
  Elapsed 29.6 s. Waiting...
  Elapsed 30.93 s. Waiting...
  Elapsed 32.26 s. Waiting...
  Elapsed 33.59 s. Waiting...
  Elapsed 34.92 s. Waiting...
  Elapsed 36.25 s. Waiting...
  Elapsed 37.59 s. Waiting...
  Elapsed 38.92 s. Waiting...
  Elapsed 40.25 s. Waiting...
  Elapsed 41.51 s. Waiting...
  Elapsed 42.81 s. Waiting...
  Elapsed 44.14 s. Waiting...
  Elapsed 45.47 s. Waiting...
  Elapsed 46.8 s. Wait

In [24]:
df_in.shape

(1273648, 3)

In [25]:
df_in.head()

,Occurrences,PageSeq_Length,PageSequence
0,1,6,/uk-residence-eu-citizens>>/uk-residence-eu-ci...
1,1,6,/government/news/operation-brock-ready-for-act...
2,1,5,/government/organisations/department-for-inter...
3,1,5,/guidance/passport-rules-for-travel-to-europe-...
4,1,9,/government/publications/statutory-instruments...


### Explore occurrences stats

In [26]:
df_in.Occurrences.describe()

count    1.273648e+06
mean     2.013853e+00
std      6.804226e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      4.351100e+04
Name: Occurrences, dtype: float64

In [27]:
all_occ = df_in.Occurrences.sum()
num_one_off_journeys = df_in[df_in.Occurrences==1].shape[0]
one_off_occ = df_in[df_in.Occurrences==1].Occurrences.sum()

In [28]:
all_occ, num_one_off_journeys, one_off_occ

(2564940, 1178335, 1178335)

In [29]:
df_in.sort_values("Occurrences", ascending=False).head()

,Occurrences,PageSeq_Length,PageSequence
1122590,43511,2,/guidance/international-driving-permits-for-uk...
1029376,29939,2,/guidance/international-driving-permits-for-uk...
1052003,23144,2,/prepare-eu-exit>>/visit-europe-brexit
3700,19203,2,/guidance/passport-rules-for-travel-to-europe-...
1222349,15202,2,/settled-status-eu-citizens-families>>/settled...


### Add `Page_List` column

In [30]:
pagelist = [pageseq.split(">>") for pageseq in df_in['PageSequence'].values]
df_in['Page_List'] = pagelist

In [31]:
df_in['Page_List'].head()

0    [/uk-residence-eu-citizens, /uk-residence-eu-c...
1    [/government/news/operation-brock-ready-for-ac...
2    [/government/organisations/department-for-inte...
3    [/guidance/passport-rules-for-travel-to-europe...
4    [/government/publications/statutory-instrument...
Name: Page_List, dtype: object

In [32]:
page_views = {}
for tup in df_in.itertuples():
    for p in tup.Page_List:
        if p in page_views.keys():
            page_views[p] += tup.Occurrences
        else:
            page_views[p] = tup.Occurrences

In [33]:
len(page_views), sum(page_views.values())

(417652, 16549470)

In [39]:
df_in.shape

(1273648, 4)

### Save out 

In [34]:
date_range = "mar_02_mar_30"
bq_dir = os.path.join(os.getenv("DATA_DIR"), "bq_journey_extract")
bq_file = os.path.join(bq_dir, "pageseq_user_journey_brexit_"+date_range+".csv.gz")
bq_file_doo = os.path.join(bq_dir, "pageseq_user_journey_brexit_"+date_range+"_doo.csv.gz")
page_views_file = os.path.join(bq_dir, "pageviews_brexit_"+date_range+".csv.gz")

In [35]:
df_in.to_csv(bq_file, compression="gzip", sep='\t', index=False)

In [36]:
df_in[df_in.Occurrences>1].to_csv(bq_file_doo, compression="gzip", sep='\t', index=False)

In [37]:
import gzip
with gzip.open(page_views_file, "wb") as writer:
    writer.write("page_url\tviews\n".encode())
    for key,value in page_views.items():
        writer.write("{}\t{}\n".format(key,value).encode())